# DNB Match-Analyse

**Zweck:** Analyse warum DNB-Abgleiche fehlschlagen

## Fragestellungen
1. Warum werden DNB-Matches abgelehnt?
2. Welche Muster gibt es bei nicht gefundenen Records?
3. Wie können wir die Match-Rate verbessern?

## Analyse-Bereiche
- Abgelehnte DNB-Matches (KI-Entscheidung: falsches Buch)
- Nicht gefundene Records bei ISBN/ISSN-Suche
- Nicht gefundene Records bei Titel/Autor-Suche
- Vergleich der Datenqualität zwischen erfolgreichen und fehlgeschlagenen Matches

In [ ]:
# Setup und Imports
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from collections import Counter

# Project setup
project_root = Path.cwd()
while not (project_root / 'config.yaml').exists() and project_root.parent != project_root:
    project_root = project_root.parent
sys.path.insert(0, str(project_root / 'src'))

from utils.notebook_utils import setup_notebook
project_root, config = setup_notebook()

# Paths
processed_dir = config.project_root / config.get('paths.data.vdeh.processed')

print(f"Project: {config.get('project.name')}")
print(f"Data: {processed_dir}")

In [ ]:
# Daten laden
print("Lade Daten...\n")

# Fused data (mit KI-Entscheidungen)
df_fused = pd.read_parquet(processed_dir / '05_fused_data.parquet')

# Original data (vor Enrichment)
df_original = pd.read_parquet(processed_dir / '03_language_detected_data.parquet')

# DNB Raw data (ISBN/ISSN Queries)
df_dnb_raw = pd.read_parquet(processed_dir / 'dnb_raw_data.parquet')

# DNB Title/Author data
df_dnb_ta = pd.read_parquet(processed_dir / 'dnb_title_author_data.parquet')

print(f"Fused: {len(df_fused):,} records")
print(f"Original: {len(df_original):,} records")
print(f"DNB ISBN/ISSN: {len(df_dnb_raw):,} queries")
print(f"DNB Titel/Autor: {len(df_dnb_ta):,} queries")

## 1. Übersicht: Wo schlägt der Abgleich fehl?

In [ ]:
# Übersicht der Fehlschläge
print("=" * 60)
print("ÜBERSICHT: FEHLGESCHLAGENE ABGLEICHE")
print("=" * 60)

# 1. DNB ISBN/ISSN nicht gefunden
isbn_not_found = (~df_dnb_raw['dnb_found']).sum()
isbn_total = len(df_dnb_raw)
print(f"\n1. ISBN/ISSN-Suche nicht gefunden: {isbn_not_found:,} / {isbn_total:,} ({isbn_not_found/isbn_total*100:.1f}%)")

# 2. DNB Titel/Autor nicht gefunden
ta_not_found = (~df_dnb_ta['dnb_found']).sum()
ta_total = len(df_dnb_ta)
print(f"2. Titel/Autor-Suche nicht gefunden: {ta_not_found:,} / {ta_total:,} ({ta_not_found/ta_total*100:.1f}%)")

# 3. KI hat DNB-Match abgelehnt
if 'fusion_title_source' in df_fused.columns:
    rejected = (df_fused['fusion_title_source'] == 'vdeh').sum()
    fusion_total = df_fused['fusion_title_source'].notna().sum()
    print(f"3. KI-Ablehnung (falscher Match): {rejected:,} / {fusion_total:,} ({rejected/fusion_total*100:.1f}%)")

# Gesamt
print(f"\n" + "=" * 60)
total_records = len(df_original)
no_dnb_data = total_records - fusion_total
print(f"Records ohne DNB-Daten: {no_dnb_data:,} / {total_records:,} ({no_dnb_data/total_records*100:.1f}%)")

## 2. Analyse: Abgelehnte DNB-Matches

Warum hat die KI entschieden, dass der DNB-Treffer nicht zum VDEH-Record passt?

In [ ]:
# Abgelehnte Matches analysieren
rejected_records = df_fused[df_fused['fusion_title_source'] == 'vdeh'].copy()

print(f"Abgelehnte DNB-Matches: {len(rejected_records):,}")
print("\n" + "=" * 60)

# Ablehnungsgründe analysieren
if 'fusion_rejection_reason' in rejected_records.columns:
    print("\nAblehnungsgründe (aus KI-Reasoning):")
    print("-" * 40)
    
    # Sammle alle Gründe
    reasons = rejected_records['fusion_rejection_reason'].dropna()
    
    if len(reasons) > 0:
        # Zeige einige Beispiele
        print(f"\nBeispiele ({min(10, len(reasons))} von {len(reasons)}):")
        for i, reason in enumerate(reasons.head(10)):
            print(f"\n{i+1}. {reason[:200]}..." if len(str(reason)) > 200 else f"\n{i+1}. {reason}")
    else:
        print("Keine expliziten Ablehnungsgründe gespeichert.")

In [ ]:
# Detaillierte Beispiele von abgelehnten Matches
print("\n" + "=" * 60)
print("BEISPIELE: Abgelehnte DNB-Matches")
print("=" * 60)

# Zeige 5 zufällige Beispiele
sample_rejected = rejected_records.sample(min(5, len(rejected_records)), random_state=42)

for idx, row in sample_rejected.iterrows():
    print(f"\n{'─' * 60}")
    print(f"Record ID: {idx}")
    print(f"\nVDEH-Daten:")
    print(f"  Titel: {row.get('title', 'N/A')}")
    print(f"  Autor: {row.get('authors_str', 'N/A')}")
    print(f"  Jahr: {row.get('year', 'N/A')}")
    print(f"  Publisher: {row.get('publisher', 'N/A')}")
    
    # DNB-Daten (die abgelehnt wurden)
    print(f"\nDNB-Daten (abgelehnt):")
    if pd.notna(row.get('dnb_title')):
        print(f"  Titel: {row.get('dnb_title', 'N/A')}")
        print(f"  Autor: {row.get('dnb_authors', 'N/A')}")
        print(f"  Jahr: {row.get('dnb_year', 'N/A')}")
        print(f"  Publisher: {row.get('dnb_publisher', 'N/A')}")
    elif pd.notna(row.get('dnb_title_ta')):
        print(f"  Titel: {row.get('dnb_title_ta', 'N/A')}")
        print(f"  Autor: {row.get('dnb_authors_ta', 'N/A')}")
        print(f"  Jahr: {row.get('dnb_year_ta', 'N/A')}")
        print(f"  Publisher: {row.get('dnb_publisher_ta', 'N/A')}")
    
    # KI-Reasoning
    if pd.notna(row.get('fusion_ai_reasoning')):
        reasoning = str(row['fusion_ai_reasoning'])[:300]
        print(f"\nKI-Begründung: {reasoning}..." if len(str(row['fusion_ai_reasoning'])) > 300 else f"\nKI-Begründung: {reasoning}")

## 3. Analyse: ISBN/ISSN nicht in DNB gefunden

Warum werden manche ISBN/ISSN nicht in der DNB gefunden?

In [ ]:
# ISBN/ISSN nicht gefunden analysieren
not_found_isbn = df_dnb_raw[~df_dnb_raw['dnb_found']].copy()

print(f"ISBN/ISSN nicht gefunden: {len(not_found_isbn):,}")
print("\n" + "=" * 60)

# Merge mit Original-Daten für mehr Kontext
not_found_with_context = not_found_isbn.merge(
    df_original[['title', 'authors_str', 'year', 'publisher', 'isbn', 'issn']],
    left_index=True, right_index=True, how='left'
)

# Analyse nach Jahr
print("\nVerteilung nach Erscheinungsjahr:")
print("-" * 40)
year_dist = not_found_with_context['year'].dropna().astype(int)
if len(year_dist) > 0:
    # Dekaden
    decades = (year_dist // 10 * 10).value_counts().sort_index()
    for decade, count in decades.tail(10).items():
        print(f"  {int(decade)}er: {count:,}")
    
    print(f"\n  Median Jahr: {int(year_dist.median())}")
    print(f"  Ältestes: {int(year_dist.min())}")
    print(f"  Neuestes: {int(year_dist.max())}")

In [ ]:
# ISBN-Format analysieren
print("\n" + "=" * 60)
print("ISBN-FORMAT ANALYSE")
print("=" * 60)

# ISBN-Längen
isbn_values = not_found_with_context['isbn'].dropna().astype(str)

if len(isbn_values) > 0:
    # Längenverteilung
    lengths = isbn_values.str.replace('-', '').str.replace(' ', '').str.len()
    print("\nISBN-Längen (ohne Bindestriche):")
    for length, count in lengths.value_counts().sort_index().items():
        print(f"  {length} Zeichen: {count:,}")
    
    # Beispiele für ungewöhnliche ISBNs
    unusual = isbn_values[lengths < 10]
    if len(unusual) > 0:
        print(f"\nBeispiele für kurze ISBNs ({len(unusual)} Stück):")
        for isbn in unusual.head(10):
            print(f"  '{isbn}'")
    
    # ISBNs die mit 3 beginnen (deutsche ISBNs)
    german_isbn = isbn_values[isbn_values.str.replace('-', '').str.startswith('3')]
    print(f"\nDeutsche ISBNs (beginnen mit 3): {len(german_isbn):,} ({len(german_isbn)/len(isbn_values)*100:.1f}%)")

In [ ]:
# Beispiele für nicht gefundene ISBN
print("\n" + "=" * 60)
print("BEISPIELE: ISBN nicht in DNB gefunden")
print("=" * 60)

# Zeige Beispiele
sample_not_found = not_found_with_context.sample(min(10, len(not_found_with_context)), random_state=42)

for idx, row in sample_not_found.iterrows():
    print(f"\n{'─' * 50}")
    print(f"ISBN: {row.get('isbn', 'N/A')}")
    print(f"ISSN: {row.get('issn', 'N/A')}")
    print(f"Titel: {row.get('title', 'N/A')}")
    print(f"Autor: {row.get('authors_str', 'N/A')}")
    print(f"Jahr: {row.get('year', 'N/A')}")
    print(f"Publisher: {row.get('publisher', 'N/A')}")

## 4. Analyse: Titel/Autor-Suche nicht gefunden

Warum findet die Titel/Autor-Suche keinen Treffer?

In [ ]:
# Titel/Autor nicht gefunden analysieren
not_found_ta = df_dnb_ta[~df_dnb_ta['dnb_found']].copy()

print(f"Titel/Autor nicht gefunden: {len(not_found_ta):,}")
print("\n" + "=" * 60)

# Merge mit Original-Daten
not_found_ta_context = not_found_ta.merge(
    df_original[['title', 'authors_str', 'year', 'publisher', 'lang_code']],
    left_index=True, right_index=True, how='left'
)

# Analyse nach Sprache
print("\nVerteilung nach Sprache:")
print("-" * 40)
if 'lang_code' in not_found_ta_context.columns:
    lang_dist = not_found_ta_context['lang_code'].value_counts()
    for lang, count in lang_dist.head(10).items():
        pct = count / len(not_found_ta_context) * 100
        print(f"  {lang}: {count:,} ({pct:.1f}%)")

# Titel-Länge analysieren
print("\nTitel-Länge:")
print("-" * 40)
title_lengths = not_found_ta_context['title'].dropna().str.len()
if len(title_lengths) > 0:
    print(f"  Min: {title_lengths.min()}")
    print(f"  Max: {title_lengths.max()}")
    print(f"  Median: {title_lengths.median():.0f}")
    print(f"  Sehr kurze Titel (<20 Zeichen): {(title_lengths < 20).sum():,}")
    print(f"  Sehr lange Titel (>200 Zeichen): {(title_lengths > 200).sum():,}")

In [ ]:
# Autor-Probleme analysieren
print("\n" + "=" * 60)
print("AUTOR-ANALYSE")
print("=" * 60)

# Fehlende Autoren
missing_author = not_found_ta_context['authors_str'].isna() | (not_found_ta_context['authors_str'] == '')
print(f"\nRecords ohne Autor: {missing_author.sum():,} ({missing_author.sum()/len(not_found_ta_context)*100:.1f}%)")

# Autor-Format analysieren
authors = not_found_ta_context['authors_str'].dropna()
authors = authors[authors != '']

if len(authors) > 0:
    # Anzahl Autoren pro Record
    author_counts = authors.str.count(';') + 1
    print(f"\nAnzahl Autoren pro Record:")
    print(f"  1 Autor: {(author_counts == 1).sum():,}")
    print(f"  2-3 Autoren: {((author_counts >= 2) & (author_counts <= 3)).sum():,}")
    print(f"  >3 Autoren: {(author_counts > 3).sum():,}")
    
    # Beispiele für problematische Autorennamen
    print(f"\nBeispiele für Autorennamen:")
    for author in authors.sample(min(5, len(authors)), random_state=42):
        print(f"  '{author}'")

In [ ]:
# Beispiele für nicht gefundene Titel/Autor
print("\n" + "=" * 60)
print("BEISPIELE: Titel/Autor nicht in DNB gefunden")
print("=" * 60)

# Zeige Beispiele
sample_ta = not_found_ta_context.sample(min(10, len(not_found_ta_context)), random_state=42)

for idx, row in sample_ta.iterrows():
    print(f"\n{'─' * 50}")
    print(f"Titel: {row.get('title', 'N/A')}")
    print(f"Autor: {row.get('authors_str', 'N/A')}")
    print(f"Jahr: {row.get('year', 'N/A')}")
    print(f"Sprache: {row.get('lang_code', 'N/A')}")

## 5. Vergleich: Erfolgreiche vs. Fehlgeschlagene Matches

In [ ]:
# Vergleich der Datenqualität
print("=" * 60)
print("VERGLEICH: Erfolgreiche vs. Fehlgeschlagene Matches")
print("=" * 60)

# ISBN/ISSN Suche
found_isbn = df_dnb_raw[df_dnb_raw['dnb_found']]
not_found_isbn = df_dnb_raw[~df_dnb_raw['dnb_found']]

# Merge mit Original für Kontext
found_context = found_isbn.merge(df_original[['year', 'lang_code']], left_index=True, right_index=True, how='left')
not_found_context = not_found_isbn.merge(df_original[['year', 'lang_code']], left_index=True, right_index=True, how='left')

print("\nISBN/ISSN-Suche:")
print("-" * 40)

# Jahr
found_year = found_context['year'].dropna().median()
not_found_year = not_found_context['year'].dropna().median()
print(f"  Median Jahr (gefunden): {found_year:.0f}")
print(f"  Median Jahr (nicht gefunden): {not_found_year:.0f}")

# Sprache
print(f"\n  Sprache (gefunden):")
for lang, count in found_context['lang_code'].value_counts().head(3).items():
    print(f"    {lang}: {count:,}")

print(f"\n  Sprache (nicht gefunden):")
for lang, count in not_found_context['lang_code'].value_counts().head(3).items():
    print(f"    {lang}: {count:,}")

In [ ]:
# Visualisierung: Jahr-Verteilung
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ISBN/ISSN
ax = axes[0]
found_years = found_context['year'].dropna()
not_found_years = not_found_context['year'].dropna()

ax.hist([found_years, not_found_years], bins=30, label=['Gefunden', 'Nicht gefunden'], 
        alpha=0.7, color=['#2ecc71', '#e74c3c'])
ax.set_xlabel('Erscheinungsjahr')
ax.set_ylabel('Anzahl')
ax.set_title('ISBN/ISSN-Suche: Verteilung nach Jahr')
ax.legend()

# Titel/Autor
ax = axes[1]
found_ta = df_dnb_ta[df_dnb_ta['dnb_found']]
not_found_ta = df_dnb_ta[~df_dnb_ta['dnb_found']]

found_ta_context = found_ta.merge(df_original[['year']], left_index=True, right_index=True, how='left')
not_found_ta_context2 = not_found_ta.merge(df_original[['year']], left_index=True, right_index=True, how='left')

found_ta_years = found_ta_context['year'].dropna()
not_found_ta_years = not_found_ta_context2['year'].dropna()

ax.hist([found_ta_years, not_found_ta_years], bins=30, label=['Gefunden', 'Nicht gefunden'], 
        alpha=0.7, color=['#2ecc71', '#e74c3c'])
ax.set_xlabel('Erscheinungsjahr')
ax.set_ylabel('Anzahl')
ax.set_title('Titel/Autor-Suche: Verteilung nach Jahr')
ax.legend()

plt.tight_layout()
plt.show()

## 6. Muster und Verbesserungspotenzial

In [ ]:
# Muster identifizieren
print("=" * 60)
print("IDENTIFIZIERTE MUSTER")
print("=" * 60)

# 1. Alte Bücher
old_threshold = 1970
old_not_found = not_found_context[not_found_context['year'] < old_threshold]
old_found = found_context[found_context['year'] < old_threshold]

print(f"\n1. Bücher vor {old_threshold}:")
print(f"   Gefunden: {len(old_found):,}")
print(f"   Nicht gefunden: {len(old_not_found):,}")
if len(old_found) + len(old_not_found) > 0:
    success_rate = len(old_found) / (len(old_found) + len(old_not_found)) * 100
    print(f"   Erfolgsrate: {success_rate:.1f}%")

# 2. Nicht-deutsche Bücher
non_german_not_found = not_found_context[not_found_context['lang_code'] != 'de']
non_german_found = found_context[found_context['lang_code'] != 'de']

print(f"\n2. Nicht-deutschsprachige Bücher:")
print(f"   Gefunden: {len(non_german_found):,}")
print(f"   Nicht gefunden: {len(non_german_not_found):,}")
if len(non_german_found) + len(non_german_not_found) > 0:
    success_rate = len(non_german_found) / (len(non_german_found) + len(non_german_not_found)) * 100
    print(f"   Erfolgsrate: {success_rate:.1f}%")

# 3. Bücher ohne Autor (für Titel/Autor-Suche)
ta_context_full = df_dnb_ta.merge(df_original[['authors_str']], left_index=True, right_index=True, how='left')
no_author = ta_context_full['authors_str'].isna() | (ta_context_full['authors_str'] == '')
no_author_found = (ta_context_full[no_author]['dnb_found']).sum()
no_author_total = no_author.sum()

print(f"\n3. Titel/Autor-Suche ohne Autorinfo:")
print(f"   Total: {no_author_total:,}")
print(f"   Gefunden: {no_author_found:,}")
if no_author_total > 0:
    print(f"   Erfolgsrate: {no_author_found/no_author_total*100:.1f}%")

In [ ]:
# Empfehlungen
print("\n" + "=" * 60)
print("EMPFEHLUNGEN ZUR VERBESSERUNG")
print("=" * 60)

print("""
Basierend auf der Analyse:

1. ALTE BÜCHER (vor 1970)
   - DNB hat möglicherweise keine vollständige Erfassung
   - Alternative: Katalog der Deutschen Staatsbibliothek
   - Alternative: WorldCat für internationale Suche

2. NICHT-DEUTSCHE BÜCHER
   - DNB fokussiert auf deutsche Publikationen
   - Empfehlung: Library of Congress, British Library APIs
   - Empfehlung: WorldCat OCLC

3. FEHLENDE AUTOREN
   - Titel/Autor-Suche funktioniert schlecht ohne Autor
   - Empfehlung: Nur Titel-Suche mit Jahr als Filter

4. ISBN-FORMAT
   - Alte ISBN-10 Formate prüfen
   - Bindestriche und Leerzeichen normalisieren
   - Prüfsummenvalidierung vor DNB-Query

5. TITEL-NORMALISIERUNG
   - Sonderzeichen entfernen
   - Untertitel abtrennen
   - Artikelpräfixe entfernen (Der, Die, Das, The, etc.)
""")

## 7. Zusammenfassung

In [ ]:
# Zusammenfassung
print("=" * 60)
print("ZUSAMMENFASSUNG")
print("=" * 60)

print(f"""
Gesamtübersicht der Fehlschläge:

1. ISBN/ISSN nicht in DNB gefunden:
   {isbn_not_found:,} Records ({isbn_not_found/isbn_total*100:.1f}%)
   
2. Titel/Autor nicht in DNB gefunden:
   {ta_not_found:,} Records ({ta_not_found/ta_total*100:.1f}%)
   
3. DNB-Match durch KI abgelehnt:
   {rejected:,} Records ({rejected/fusion_total*100:.1f}%)

Hauptgründe für Fehlschläge:
- Alte Publikationen (vor DNB-Erfassung)
- Ausländische Publikationen (nicht in DNB)
- Fehlende/ungenaue Metadaten (Autor, ISBN)
- Titel-Variationen zwischen VDEH und DNB

Nächste Schritte:
- Alternative Datenquellen für alte/ausländische Bücher
- Verbesserte Normalisierung der Suchterme
- Fuzzy-Matching für Titel-Suche
""")